# Exercice 1 - Optimisation avec autograd

Un problème d'optimisation numérique peut être résolu par l'algorithme de descente de gradient.

La descente de gradient se base sur l'idée que si une fonction multivariée $F(\mathbf{x})$ est dérivable au voisinage du point $\mathbf{a}$, alors $F(\mathbf{x})$ diminue ''plus rapidement'' si, partant du point  $\mathbf{a}$, on suit la direction inverse du gradient de $F$ en $\mathbf{a}$, soit $-\nabla F(\mathbf{a})$. Il s'ensuit donc que si :

\begin{equation}
\mathbf{a}_{n+1} = \mathbf{a}_n-\alpha\nabla F(\mathbf{a}_n)
\end{equation}

pour $\alpha \in \mathbb{R}_{+}$ suffisamment petit ($\alpha$ est aussi appelé taux d'apprentissage), alors  $F(\mathbf{a_n})\geq F(\mathbf{a_{n+1}})$. Autrement dit, le terme $\alpha\nabla F(\mathbf{a})$ est soustrait à $\mathbf{a}$ pour se diriger vers le minimum. Partant d'un point initial $\mathbf{x}_0$, on construit donc une séquence de points visités $\mathbf{x}_0, \mathbf{x}_1, \mathbf{x}_2, \dots$ tels que :

\begin{equation}
\mathbf{x}_{n+1}=\mathbf{x}_n-\alpha \nabla F(\mathbf{x}_n),\ n \ge 0.
\end{equation}

Avec la propriété suivante :

\begin{equation}
F(\mathbf{x}_0)\ge F(\mathbf{x}_1)\ge F(\mathbf{x}_2)\ge \cdots
\end{equation}

## Application avec un exemple en dimension 2

In [6]:
# Inclure les fonction d'affichage "print"
from __future__ import print_function
# Inclure numpy abrégé np
import numpy as np
# Inclure les fonction d'affichage "pyplot" abrégée plt
from matplotlib import pyplot as plt
# Inclure torch
import torch

# %matplotlib inline

AttributeError: module 'matplotlib' has no attribute 'artist'

**Définition de la fonction "plot_with_contours" permettant de visualiser une fonction 2D -> 1D grâce à des contours**

In [ ]:
def plot_with_contours(F):
    # Pas spatial pour les vecteurs 
    delta = 0.1
    
    # Vecteurs xsteps et ysteps construits à partir du pas delta
    xsteps = np.arange(-4., 4., delta)
    ysteps = np.arange(-3., 3., delta)
    
    # Matrice 2D Z = F(x,y)
    Z = np.array([[F(torch.tensor([x, y])) for x in xsteps] for y in ysteps])
    
    # Grilles X et Y créées à partir des vecteurs xsteps et ysteps
    X, Y = np.meshgrid(xsteps, ysteps)
    
    # Affichage des lignes de niveaux (contours)
    plt.contour(X, Y, Z, levels=np.arange(Z.min(), Z.max(), 0.05 * (Z.max() - Z.min())), cmap='hot')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Lignes de niveau de F (contours)')

**Définition et affichage d'une fonction "F" un peu compliquée à minimiser**

In [ ]:
def F(x):
    # Fonction F(x,y) = sin(x+1.1) + sin(y) + 0.4*(|x|-2.5 + |y|+0.5)
    return torch.sin(x[0]+1.1) + torch.sin(x[1]) + 0.4*(torch.abs(x[0]-2.5) + torch.abs(x[1]+0.5))

# Affichage des lignes de niveaux (contours)
plot_with_contours(F)

**Ecriture de la boucle itérative de descente du gradient**

In [ ]:
# Point initial (tester différentes valeurs)
# Noter que l'on manipule des objets de type Tensor
x = torch.tensor([0., 0.])

# Taux d'apprentissage (tester différentes valeurs)
learning_rate = 0.1cd

# Nombre maximum d'itérations
it_max = 100

# Liste pour garder les différentes valeurs de la fonction objectif F à chaque itération
objectives = []

# Liste pour garder les différentes valeurs du couple (x,y) à chaque itération
points = []

# Descente de gradient à effectuer "it_max" fois
for it in range(it_max):
    
    # Convertir le Tensor en simple couple numérique de points (x,y) et l'ajouter dans la liste "points"
    points.append(x.numpy())
    
    # (R)ajouter la dépendance au gradient à "vrai" pour pouvoir calculer le gradient de F par rapport à x
    x.requires_grad = True
    
    # Appeller la fonction F en (x,y) à l'itération courante
    Z = F(x)
    
    # Afficher la valeur de F 
    # Noter que .item() permet de ne récupérer que la valeur numérique de l'objet Z qui est un Tensor 
    # ayant hérité d'une fonction permettant de calculer le gradient par rapport à x
    # Si le Tensor n'est pas pas scalaire, il faut utiliser .detach().numpy() au lieu de .item()
    print('Iteration', it+1, '> F =', "%.5f" % Z.item())
    
    # Ajouter la valeur numérique courante de F(x,y) dans "objectives"
    objectives.append(Z.item())
    
    # Calculer le gradient de F par rapport à x
    Z.backward()    
    
    # Mettre à jour la solution courante (x,y)
    # Noter que le calcul n'est possible qu'avec des Tensor qui n'ont pas de dépendance comme "requires_grad" par exemple
    # Afin d'enlever le "requires_grad", on le détache de x avec x.detach()
    # La sortie x ici n'a plus de dépendance avec "requires_grad", ce n'est plus qu'un simple Tensor
    x = x.detach() - learning_rate*x.grad

**Affichage des résultats**

In [ ]:
# Afficher les itérés de F
plt.plot(objectives)
plt.xlabel('itérations')
plt.ylabel('fonction F')

In [ ]:
# Transformer la liste "points" en tableau
points = np.array(points)

# Afficher les lignes de niveaux (contours)
plot_with_contours(F)

# Afficher les itérés des couples (x,y)
plt.plot(points[:, 0], points[:, 1], 'bo-')

**Questions**

- La solution trouvée est-elle un minimum global ou local de la fonction F ?

- Tester différents points initiaux pour $\mathbf{x}_0$.

- Tester différents taux d'apprentissage (par exemple 1 ou 0.01). Qu'observez vous ?
